# WeatherBot
##### by Anna Bialas
## General query:
##### (an example using San Marino as a location name)
@annabialas bot weather san marino
## Checking the weather by location name:
##### (lesser accuracy due to the OpenWeatherAPI choosing the location matching the name on its own)
@annabialas bot weather san marino name
## Checking the weather by location id :
##### (greater accuracy because one id has only one location)
@annabialas bot weather 3168068
## Finding the id(s) corresponding to a location name: 
##### (parses the city.list.json file to find the ids)
@annabialas bot weather san marino id

In [ ]:
import time
import re
import requests
import json

In [ ]:
def message_matches(user_id, message_text):
    '''
    Check if the username and the word 'bot' appears in the text
    '''
    regex_expression = r'.*@' + user_id + r'.*bot.*'
    regex = re.compile(regex_expression, re.IGNORECASE)
    # Check if the message text matches the regex above
    match = regex.match(message_text)
    # returns true if the match is not None (ie the regex had a match)
    return match != None 

In [ ]:
def extract_location(message_text):
    '''
    Extract the command. The regex relies on the question having a specific form
    '''
    
    # adding the optional 'in' in case people make a mistake... I know I did and I am the one who made this bot so...
    regex_expression = r'weather (in )?(\D+)'
    regex= re.compile(regex_expression, re.IGNORECASE)
    matches = regex.finditer(message_text)
    for match in matches:
        if match: 
            location = match.group(2).title()
            return location
        else:
            return None

In [ ]:
def specify_method(message_text):
    '''
    Extract the command. The regex relies on the question having a specific form
    '''
    
    #again with the in... just in case they make a mistake but it still goes through...
    regex_expression = r'weather (in )?(\D+) (id|name)'
    regex = re.compile(regex_expression, re.IGNORECASE)
    matches = regex.finditer(message_text)
    for match in matches:
        return match.group(2).title(), match.group(3)
    
    return None

In [ ]:
def get_weather_by_location(location):
    
    openweathermap_url = "http://api.openweathermap.org/data/2.5/weather"
    parameters = {
        'q'     :  location,
        'units' : 'imperial',
        'mode'  : 'json',
        'appid' : 'fc5c34780a765f2d312bd692c9e9d3a6'
    }
    
    weather = requests.get(openweathermap_url, params=parameters).json()
    
    if weather['cod'] == '404':
        return weather['message']
    else:
        weather = "*Weather in {l}:*\nAverage temperature: {t}F, {w}, humidity: {h}.".format(l=location, t=weather["main"]["temp"], w=weather["weather"][0]["description"], h=weather["main"]["humidity"])
        return weather

In [ ]:
def get_location_id(location):
    '''
    Returns a list of JSON objects with the location name and its ID
    '''
    location_file = 'city.list.json'
    f = open(location_file, 'r') 
    content = f.read()
    f.close()
    
    items = content.split("\n")
    
    matching_locations = []
    
    regex_expression = r'\b' + re.escape(location) + r'\b'
    regex = re.compile(regex_expression)
    
    for item in items:
        item_parsed = json.loads(item)
        matches = regex.finditer(item_parsed['name'])
        for match in matches:
            if match.group() in item_parsed['name']:
                matching_locations.append(item_parsed)
                
    if matching_locations != None:
        return matching_locations
    else:
        return None

In [ ]:
def list_ids(location):
    
    if location != None:
    
        message = "Thank you for asking about the weather in {l}.\n".format(l=location)

        matching_locations = get_location_id(location)
        print(matching_locations)
        
        if len(matching_locations) == 0:
            message += "Unfortunately, I could not find any IDs corresponding to {l} :disappointed: \
Check if you spelled the name of the location correctly and try again!".format(l=location)
            return message
        
        # If there is one matching location
        if len(matching_locations) == 1:
            message += "*I have found one ID corresponding to your desired location* :tada: \n"
        
        # If there are multiple matching locations
        if len(matching_locations) > 1:
            message += "*I have multiple results matching your desired location* :tada: :tada: :tada: \n"
            
        # Add the information for each location
        for entry in matching_locations:
            name = entry['name']
            country = entry['country']
            _id = entry['_id']
            lon = entry['coord']['lon']
            lat = entry['coord']['lat']
            
            message += "{n}, {c}, located at ({lon}, {lat}), has id of {_id}\n".format(n=name, c=country, lon=lon, lat=lat, _id=_id)
       
        message += "To see weather for a specific location by its ID\
 please type `weather` followed by the chosen `ID`, e.g. `weather 4833279`"
    
    else:
        message = "Unfortunately, I could not find any ids corresponding to {l} :disappointed: \
Check if you spelled the name of the location correctly and try again!".format(l=location)
        
    return message

In [ ]:
def extract_id(message_text):
    
    regex_expression = r'weather ([0-9]{7})' 
    regex = re.compile(regex_expression, re.IGNORECASE)
    matches = regex.finditer(message_text)
    for match in matches:
        return match.group(1)

    return None

In [ ]:
def get_weather_by_id(location_id):

    openweathermap_url = "http://api.openweathermap.org/data/2.5/weather"
    parameters = {
        'id'    :  location_id,
        'units' : 'imperial',
        'mode'  : 'json',
        'appid' : 'fc5c34780a765f2d312bd692c9e9d3a6'
    }
    
    weather = requests.get(openweathermap_url, params=parameters).json()
    
    if weather['cod'] == '400':
        message = 'Bad news! Unfortunately I could not find the weather for the ID you were\
 asking for :disappointed: Please, check if it was typed correctly!'
        return message
    else:
        weather = "*Weather in {n}:*\nAverage temperature: {t}F, {w}, humidity: {h}.".format(n=weather["name"], t=weather["main"]["temp"], w=weather["weather"][0]["description"], h=weather["main"]["humidity"])
        return weather

In [ ]:
def create_message(message_text, weather_type, location, location_id):
    
    if location != None:
    
        method = specify_method(message_text)
        
        if method != None:
            
            location = specify_method(message_text)[0]
            weather_type = specify_method(message_text)[1]
            
            if weather_type == 'name':
                weather = get_weather_by_location(location)
                if weather == 'city not found':
                    message = 'Thank you for asking about the weather in {l}. Unfortunately I could not find the location name you were\
 asking for :disappointed:'.format(l=location)
                else:
                    message = weather
        
            if weather_type == 'id':
                message = list_ids(location)
                
        else:
            result = get_weather_by_location(location)
            if result == 'city not found':
                message = 'Thank you for asking about the weather in {l}. Unfortunately I could not find the location you were\
 asking for :disappointed: Please, make sure you spelled the location name correctly, or try a different one!'.format(l=location)
            else:
                message = "Would you like to check the weather using the name of your requested location or its ID?\
 I recommend using the ID to ensure greatest accuracy :sunglasses:\
 If you choose to go with the name, you might want to add its state and/or country :upside_down_face:\n\
 Please type `weather [query],[state],[country] name` or `weather [query] id` to make your choice."
            
    elif location_id != None:
        message = get_weather_by_id(location_id)
  
    else: 
        message = "Sorry, I did not get that :thinking_face: \
Please try `weather` followed by `name of the location` to check for its weather.\n\
e.g. `weather chicago` or `weather paris` :upside_down_face:"
        
    
    return message

In [ ]:
# Read the access token from the file and create the Slack Client
secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()

auth_info = json.loads(content)
token = auth_info["access_token"]

from slackclient import SlackClient
sc = SlackClient(token)

In [ ]:
response = sc.api_call("users.info", user=auth_info["user_id"])
user = response['user']

In [ ]:
# For me to see if it's working
print("The username of the authenticated user is", user.get('name'))
print("The email of the authenticated user is", user.get('profile').get('email'))
print("The email of the authenticated user is", user.get('profile').get('real_name'))

In [ ]:
# Connect to the Real Time Messaging API of Slack and process the events

if sc.rtm_connect():
    # We are going to be polling the Slack API for recent events continuously
    while True:
        # We are going to wait 1 second between monitoring attempts
        time.sleep(1)
        # If there are any new events, we will get a response. If there are no events, the response will be empty
        response = sc.rtm_read()
        for item in response:
            
            # Check that the event is a message. If not, ignore and proceed to the next event.
            if item.get("type") != 'message':
                continue
                
            # Check that the message comes from a user. If not, ignore and proceed to the next event.
            if item.get("user") == None:
                continue
                
            print(item)
            
            # Check that the message is asking the bot to do something. If not, ignore and proceed to the next event.
            user_id = auth_info["user_id"]
            message_text = item.get('text')
            if not message_matches(user_id, message_text):
                continue
        
            location = extract_location(message_text)
            print('location is:', location)
            
            location_id = extract_id(message_text)
            print("location id is:", location_id)
            
            method = specify_method(message_text)
            print('method is:', method)
            
            message = create_message(message_text, method, location, location_id)

            # Post a response to the #bots channel
            sc.api_call("chat.postMessage", channel="#assignment2_bots", text=message)
            print(message)